In [7]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from selenium.webdriver.common.by import By
import json
import pygsheets
from google.oauth2 import service_account
import platform
import datetime
from dateutil.relativedelta import relativedelta
import sys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

class MyChromeDriver(webdriver.Chrome):

    def __init__(self, headless=False, maximize=True):

        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')

        webdriver.Chrome.__init__(self, service=ChromeService(ChromeDriverManager().install()), options=options)

        if maximize:
            self.maximize_window()

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

if driver is None:
        my_driver = MyChromeDriver()
        time.sleep(10)
else:
    my_driver = driver

os.getcwd()

'c:\\Users\\SANDBOX\\Desktop\\Sandbox\\crawling'

In [8]:
# # 크롬 드라이버 옵션 설정 : 지정하면 크롬 아예 안보임
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(options=chrome_options)

# 구글 시트 가져오기
gc = pygsheets.authorize(service_account_file='creds.json')
sheetname= '크롤링 시도'
sh = gc.open(sheetname)
wks_tag = sh.worksheet_by_title('전체')
ncv_df = wks_tag.get_as_df(has_header=True , index_column=None , start='A1', end='AZ100000' , numerize=True , empty_value=np.nan)
ncv_df = ncv_df[ncv_df['단가 기준'] >= '2023-09']
ncv_df = ncv_df[ncv_df['회사'] != 'Sandbox']
ncv_df = ncv_df[ncv_df['평균 조회수(NCV)'].isnull()]
len(ncv_df)

1944

In [10]:
# 크롤링
channel_id_list = ncv_df['채널 ID'].unique().tolist()
youtube_video_url_dic = {}

driver = webdriver.Chrome() # 크롬 옵션 지정 시 options=chrome_options 추가

for i in tqdm(range(len(channel_id_list))):
    url = 'https://www.youtube.com/channel/' + channel_id_list[i] + '/videos'
    print(i, url)

    driver.get(url)
    time.sleep(3)

    pageString = driver.page_source
    soup = BeautifulSoup(pageString, features='html.parser')

    date_count = 0
    video_date_list = []
    video_view_list = []
    date_list = ['분 전', '시간 전', '1일 전', '2일 전', '3일 전', '4일 전', '5일 전', '6일 전', '7일 전', '8일 전', '9일 전',
                 '10일 전', '11일 전', '12일 전', '13일 전', '2주 전', '3주 전', '4주 전', '1개월 전', '2개월 전', '3개월 전']

    # 날짜, 제목
    date_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[1::2]
    title_soup = soup.find_all('yt-formatted-string', attrs={'class' : 'style-scope ytd-rich-grid-media'})
    view_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[0::2]

    # 날짜 기준에 맞는 영상 개수 카운트
    for j in range(len(date_soup)):
        if date_soup[j].text in date_list:
            date_count += 1
        if date_list[0] in date_soup[j].text:
            date_count += 1

    # 날짜 기준에 맞도록 날짜, 제목 정보 저장
    date_soup = date_soup[:date_count]
    title_soup = title_soup[:date_count]
    view_soup = view_soup[:date_count]

    # 영상 url 저장
    for k in range(len(date_soup)):
        video_date = date_soup[k].text
        video_date_list.append(video_date)

        video_view = view_soup[k].text.replace('조회수 ', '').replace('회', '')
        video_view_list.append(video_view)

        print(video_date, video_view)

    youtube_video_url_dic[channel_id_list[i]] = video_date_list
    youtube_video_url_dic[channel_id_list[i]] = video_view_list
    
driver.quit()


  0%|          | 0/1936 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCe8YEoKunUWyVQQIGeGa-Dg/videos


  0%|          | 1/1936 [00:06<3:45:35,  7.00s/it]

3일 전 4.8천
11일 전 4.7천
4주 전 5.4천
1개월 전 8천
1개월 전 6.5천
1개월 전 5.3천
2개월 전 8.2천
2개월 전 8.9천
2개월 전 1.3만
2개월 전 8.3천
3개월 전 9.1천
3개월 전 9.4천
3개월 전 2만
3개월 전 1.7만
3개월 전 1.8만
3개월 전 9.7천
1 https://www.youtube.com/channel/UCWCldx-uKM2wJWVuWk9SAHQ/videos


  0%|          | 2/1936 [00:14<3:49:33,  7.12s/it]

...

 29%|██▉       | 570/1936 [49:49<1:56:34,  5.12s/it]

2일 전 1천
9일 전 1.7천
2주 전 1.9천
3주 전 2.3천
4주 전 2.7천
1개월 전 8.9천
1개월 전 1천
1개월 전 1.5천
1개월 전 1.3천
2개월 전 4.2만
2개월 전 3.8천
2개월 전 2천
2개월 전 1.6만
2개월 전 1.7천
3개월 전 4.1만
3개월 전 8.1천
3개월 전 1.3만
3개월 전 1.2만
570 https://www.youtube.com/channel/UCW5FEe1eiOAKzcnhmeI0Mzw/videos


 29%|██▉       | 571/1936 [50:48<2:01:28,  5.34s/it]

3주 전 2.7천
1개월 전 1.9천
571 https://www.youtube.com/channel/UCZTPmBCj8BLhn0DpzGaz4gA/videos


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF6080252A2+57122]
	(No symbol) [0x00007FF607F9EA92]
	(No symbol) [0x00007FF607E6E3AB]
	(No symbol) [0x00007FF607E5BA47]
	(No symbol) [0x00007FF607E5B6C0]
	(No symbol) [0x00007FF607E6FA71]
	(No symbol) [0x00007FF607EDE27F]
	(No symbol) [0x00007FF607EC6DB3]
	(No symbol) [0x00007FF607E9D2B1]
	(No symbol) [0x00007FF607E9E494]
	GetHandleVerifier [0x00007FF6082CEF82+2849794]
	GetHandleVerifier [0x00007FF608321D24+3189156]
	GetHandleVerifier [0x00007FF60831ACAF+3160367]
	GetHandleVerifier [0x00007FF6080B6D06+653702]
	(No symbol) [0x00007FF607FAA208]
	(No symbol) [0x00007FF607FA62C4]
	(No symbol) [0x00007FF607FA63F6]
	(No symbol) [0x00007FF607F967A3]
	BaseThreadInitThunk [0x00007FFF13F226AD+29]
	RtlUserThreadStart [0x00007FFF15C6AA68+40]


In [11]:
youtube_all_video_df = pd.DataFrame()
for i in range(len(youtube_video_url_dic.keys())):
    new_youtube_df = pd.DataFrame({'channel id':[list(youtube_video_url_dic.keys())[i]] * len(youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]),
                                  'view': youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]})
    youtube_all_video_df = pd.concat([youtube_all_video_df, new_youtube_df])
youtube_all_video_df = youtube_all_video_df.reset_index(drop=True)

# 채널id 고유값 개수 확인
youtube_all_video_df['channel id'].nunique()

270

In [12]:
# 조회수 계산
for i in range(len(youtube_all_video_df)):
    if '천' in str(youtube_all_video_df['view'][i]): # 1,000
        k_num = int(float(youtube_all_video_df['view'][i].replace('천', ''))*1000)
        youtube_all_video_df.loc[i, 'view'] = format(k_num)
    elif '만' in str(youtube_all_video_df['view'][i]): # 10,000
        m_num = int(float(youtube_all_video_df['view'][i].replace('만', ''))*10000)
        youtube_all_video_df.loc[i, 'view'] = format(m_num)

youtube_all_video_df_copy = youtube_all_video_df.copy()
youtube_all_video_df_copy

,channel id,view
0,UCe8YEoKunUWyVQQIGeGa-Dg,4800
1,UCe8YEoKunUWyVQQIGeGa-Dg,4700
2,UCe8YEoKunUWyVQQIGeGa-Dg,5400
3,UCe8YEoKunUWyVQQIGeGa-Dg,8000
4,UCe8YEoKunUWyVQQIGeGa-Dg,6500
...,...,...
4359,UC-6DLwseTZG-l4JprVV2MIQ,8100
4360,UC-6DLwseTZG-l4JprVV2MIQ,13000
4361,UC-6DLwseTZG-l4JprVV2MIQ,12000
4362,UCW5FEe1eiOAKzcnhmeI0Mzw,2700


In [13]:
youtube_all_video_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel id  4364 non-null   object
 1   view        4364 non-null   object
dtypes: object(2)
memory usage: 68.3+ KB


In [14]:
# 조회수(view) 문자열 -> 숫자
youtube_all_video_df_copy['view'] = pd.to_numeric(youtube_all_video_df_copy['view'], errors='coerce')
youtube_all_video_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel id  4364 non-null   object
 1   view        4364 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 68.3+ KB


In [15]:
# null 값 삭제
youtube_all_video_df_copy.dropna(inplace=True)
youtube_all_video_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel id  4364 non-null   object
 1   view        4364 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 68.3+ KB


In [16]:
# 채널별 평균 조회수 계산
channel_avg_view_df = youtube_all_video_df_copy.groupby('channel id')['view'].mean().reset_index()
channel_avg_view_df
# 데이터프레임 확인 후 구글 시트에 저장!

,channel id,view
0,UC-6DLwseTZG-l4JprVV2MIQ,8994.444444
1,UC-AuoahqpeL4yoK4kvu___g,150000.000000
2,UC-Rr7zYKnRqa_aEDn1KjVkw,8200.000000
3,UC0IsFqJzKJLRjHIznNNdhxQ,54216.666667
4,UC0wo7SwsC5dE0nSgBs2M-3A,117000.000000
...,...,...
265,UCysSC6UjONpWKMbBupJZjpg,53433.333333
266,UCyw-SlgJoR9KDaE6TWeYI-A,15808.333333
267,UCzDkPraY61LFGybv_1tGUvQ,18250.000000
268,UCzMW83VpZUg5MUc12iFyDiw,5366.666667


In [17]:
channel_avg_view_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   channel id  270 non-null    object 
 1   view        270 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.3+ KB


In [ ]:
# 결과 저장
gc = pygsheets.authorize(service_account_file='creds.json')
sheetname= '크롤링 시도'

# 오류 발생 시, worksheet의 행 수 늘려서 재실행
sh = gc.open(sheetname)
wks = sh.worksheet_by_title('유튜브 평균 조회수')
wks.set_dataframe(channel_avg_view_df, 'A1', index=False)